# Step 4: Search, Retrieval, and RAG

This is the final and most exciting step. We will now query our vector database to find relevant code, and then use those results with a Large Language Model (LLM) to generate a direct, human-readable answer. This is the complete end-to-end **Retrieval-Augmented Generation (RAG)** process.

## Prerequisites

1.  **Qdrant is Running:** Ensure your Qdrant Docker container is running via `docker-compose up -d`.
2.  **Collection is Indexed:** You should have already run the previous notebook to index the data into the `semantic_code_search` collection.

In [1]:
# Install necessary libraries, including 'rich' for pretty printing
%pip install openai python-dotenv qdrant-client rich

  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [rich]2/3 [rich]
Note: you may need to restart the kernel to use updated packages.


## 1. Setup Clients and API Key

As before, we'll start by setting up our clients for OpenAI and Qdrant. Make sure your `.env` file is in the same directory.

In [2]:
import os
import getpass
from openai import OpenAI
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.syntax import Syntax

# --- Initialize Rich Console for pretty printing ---
console = Console()

# --- OpenAI Client Setup ---
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    api_key = getpass.getpass("OpenAI API key not found. Please enter your key: ")

try:
    openai_client = OpenAI(api_key=api_key)
    console.print("✅ OpenAI client initialized successfully!")
except Exception as e:
    console.print(f"❌ Error initializing OpenAI client: {e}")

# --- Qdrant Client Setup ---
try:
    qdrant_client = QdrantClient("localhost", port=6333)
    console.print("✅ Qdrant client connected successfully!")
except Exception as e:
    console.print(f"❌ Error connecting to Qdrant: {e}")

✅ OpenAI client initialized successfully!

✅ Qdrant client connected successfully!

## 2. Define the Search Function

We'll create a function that encapsulates the search process: taking a user's text query, embedding it, and searching Qdrant for the most similar code chunks.

In [5]:
COLLECTION_NAME = "semantic_code_search"

# NOTE: qdrant_client.search is deprecated. We transparently prefer query_points when available.
def get_openai_embedding(text: str, model: str = "text-embedding-3-small"):
    text = text.replace("\n", " ")
    try:
        return openai_client.embeddings.create(input=[text], model=model).data[0].embedding
    except Exception as e:
        console.print(f"❌ Error generating embedding: {e}")
        return None


def search_code(query: str, top_k: int = 3):
    """Search for code chunks based on a natural language query.

    Uses the newer query_points API when present, else falls back to deprecated search for older client versions.
    Returns a list of ScoredPoint objects (uniform interface)."""
    query_embedding = get_openai_embedding(query)
    if not query_embedding:
        return []

    try:
        # Preferred modern API
        if hasattr(qdrant_client, "query_points"):
            result = qdrant_client.query_points(
                collection_name=COLLECTION_NAME,
                query=query_embedding,  # vector search by raw embedding
                limit=top_k,
                with_payload=True,
                with_vectors=False,
            )
            # query_points returns a models.QueryResponse; extract .points
            search_results = result.points
        else:
            # Backward compatibility path (deprecated)
            search_results = qdrant_client.search(
                collection_name=COLLECTION_NAME,
                query_vector=query_embedding,
                limit=top_k,
                with_payload=True,
            )
        return search_results
    except Exception as e:
        console.print(f"❌ Search failed: {e}")
        return []

console.print("✓ Search functions defined (using query_points API if available).")

✓ Search functions defined (using query_points API if available).

## 3. Perform the Search

Let's define our query and execute the search. We'll then look at the raw results.

In [6]:
user_query = "How do I validate an email address?"
console.print(f"[bold cyan]Query:[/bold cyan] {user_query}\n")

search_results = search_code(user_query)

console.print(f"Found {len(search_results)} relevant results.")

Query: How do I validate an email address?

Found 3 relevant results.

### Method 1: Simple Retrieval Results

First, let's just display the results directly. This is a basic semantic search implementation.

In [7]:
console.print(Panel("[bold green]Method 1: Simple Retrieval Results[/bold green]"))

for i, result in enumerate(search_results):
    payload = result.payload
    score = result.score
    
    # Create a syntax-highlighted code block
    code_snippet = Syntax(payload['snippet'], "python", theme="monokai", line_numbers=True)
    
    # Create a panel for each result
    result_panel = Panel(
        code_snippet,
        title=f"[bold]Result {i+1} | Score: {score:.4f}[/bold]",
        subtitle=f"[cyan]{payload['context']['file_path']}[/cyan]",
        border_style="blue"
    )
    console.print(result_panel)
    console.print(f"[bold]LLM Description:[/bold] {payload['llm_description']}\n")

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Method 1: Simple Retrieval Results                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Result 1 | Score: 0.6565 ────────────────────────────────────────────╮
│   1 def validate_email(email: str) -> bool:                                                                     │
│   2     """Simple email validation function."""                                                                 │
│   3     return "@" in email and "." in email.split("@")[-1]                                                     │
╰──────────────────────────────────────────── src/utils/validators.py ────────────────────────────────────────────╯

LLM Description: Performs a simple validation of an email address by checking for the presence of “@” and “.” 
symbols.

╭─────────────────────────────────────────── Result 2 | Score: 0.0299 ────────────────────────────────────────────╮
│   1 class DataProcessor:                                                                                        │
│   2     """A class for processing and analyzing data."""                                                        │
│   3                                                                                                             │
│   4     def __init__(self, data_source: str):                                                                   │
│   5         self.data_source = data_source                                                                      │
│   6         self.processed_count = 0                                                                            │
│   7                                                                                                             │
│   8     def process_batch(self, items: List[str]) -> List[str]:                                                 │
│   9         #... (rest of class code)                                                                           │
╰──────────────────────────────────────────── src/processing/core.py ─────────────────────────────────────────────╯

LLM Description: A class designed to process and analyze batches of string data, including cleaning and calculating
statistics.

╭─────────────────────────────────────────── Result 3 | Score: 0.0277 ────────────────────────────────────────────╮
│   1 def read_file_content(filepath: str) -> str:                                                                │
│   2     """Read and return the content of a file."""                                                            │
│   3     try:                                                                                                    │
│   4         with open(filepath, 'r', encoding='utf-8') as file:                                                 │
│   5             return file.read()                                                                              │
│   6     except FileNotFoundError:                                                                               │
│   7         return ""                                                                                           │
╰─────────────────────────────────────────── src/utils/file_handler.py ───────────────────────────────────────────╯

LLM Description: Reads and returns the entire content of a specified file, handling file-not-found errors 
gracefully.

### Method 2: Generating an Answer with RAG

Now, let's implement the more advanced RAG approach. We'll take the retrieved results, use them as context in a prompt for a powerful chat model (`gpt-4o`), and get a direct, synthesized answer.

In [8]:
console.print(Panel("[bold green]Method 2: Generating an Answer with RAG[/bold green]"))

# 1. Context Collection
context = ""
for result in search_results:
    payload = result.payload
    context += f"File: {payload['context']['file_path']}\nCode Snippet:\n```python\n{payload['snippet']}\n```\n\n"

# 2. Intelligent Prompting
system_prompt = "You are an expert programming assistant. Answer the user's question based ONLY on the provided context from their codebase."

human_prompt = f"""--- CONTEXT FROM THE CODEBASE ---
{context}
--- QUESTION ---
{user_query}
"""

console.print(f"[bold]System Prompt:[/bold] {system_prompt}")
console.print(f"[bold]Human Prompt Preview:[/bold] {human_prompt[:200]}...\n")

# 3. Synthesized Answer
try:
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": human_prompt}
        ],
        temperature=0.0 # Make the output deterministic
    )
    answer = response.choices[0].message.content
    
    # Display the final answer as formatted markdown
    console.print(Panel(Markdown(answer), title="[bold magenta]Synthesized Answer[/bold magenta]"))

except Exception as e:
    console.print(f"❌ Error generating RAG answer: {e}")

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Method 2: Generating an Answer with RAG                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

System Prompt: You are an expert programming assistant. Answer the user's question based ONLY on the provided 
context from their codebase.

Human Prompt Preview: --- CONTEXT FROM THE CODEBASE ---
File: src/utils/validators.py
Code Snippet:
```python
def validate_email(email: str) -> bool:
    """Simple email validation function."""
    return "@" in email and ...

╭────────────────────────────────────────────── Synthesized Answer ───────────────────────────────────────────────╮
│ To validate an email address in your codebase, you can use the validate_email function defined in the           │
│ src/utils/validators.py file. This function checks if the email contains an "@" symbol and a "." in the domain  │
│ part of the email address. Here's how you can use it:                                                           │
│                                                                                                                 │
│                                                                                                                 │
│  from src.utils.validators import validate_email                                                                │
│                                                                                                                 │
│  email = "example@example.com"                                                                                  │
│  is_valid = validate_email(email)                                                                               │
│                                                                                                                 │
│  if is_valid:                                                                                                   │
│      print("The email address is valid.")                                                                       │
│  else:                                                                                                          │
│      print("The email address is invalid.")                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│ This function will return True if the email address is considered valid based on the simple criteria it checks, │
│ and False otherwise.                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯